<a href="https://colab.research.google.com/github/VietAnhLee/NLP_1920_FinalExam/blob/master/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/Colab Notebooks/NLP_1920_FinalExam/preprocessed_df.csv" "preprocessed_df.csv"

In [0]:
from gensim.models import Word2Vec
import os
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('preprocessed_df.csv', sep='\t')

In [0]:
df

,sentence,type,point
0,áo bao đẹp ạ positive,POS,5
1,tuyệt vời positive,POS,5
2,2day ao không giong trong,NEG,1
3,mùi thơm bôi lên da mềm da nagative,POS,5
4,vải đẹp dày dặn positive,POS,5
...,...,...,...
31455,không đáng tiền,NEG,1
31456,quần rất đẹp positive,POS,5
31457,hàng đẹp đúng giá tiền positive positive,POS,5
31458,chất vải khá ổn positive,POS,4


In [0]:
# Try predict only POS, NEU and NEG
df2 = df[['sentence', 'type']].copy()

In [0]:
df2

,sentence,type
0,áo bao đẹp ạ positive,POS
1,tuyệt vời positive,POS
2,2day ao không giong trong,NEG
3,mùi thơm bôi lên da mềm da nagative,POS
4,vải đẹp dày dặn positive,POS
...,...,...
31455,không đáng tiền,NEG
31456,quần rất đẹp positive,POS
31457,hàng đẹp đúng giá tiền positive positive,POS
31458,chất vải khá ổn positive,POS


In [0]:
Dict = {'NEG': '1', 'NEU': '2', 'POS': '3'}
for i in range(len(df2)):
  df2.loc[i, 'type'] = Dict[df2.loc[i, 'type']]  

In [0]:
df2.head()

,sentence,type
0,áo bao đẹp ạ positive,3
1,tuyệt vời positive,3
2,2day ao không giong trong,1
3,mùi thơm bôi lên da mềm da nagative,3
4,vải đẹp dày dặn positive,3


In [0]:
n = int(len(df2) * 0.3)

In [0]:
train = df2[n:]
test = df2[:n]

In [0]:
len(train) + len(test)

31460

In [0]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [0]:
# using tf-idf with 1 - 3 ngram range
tfidf = TfidfVectorizer(
    min_df = 3, 
    max_df = 0.5, 
    ngram_range=(1,3),
    use_idf=True,
    sublinear_tf=True
)

In [0]:
# Have to add .values.astype('U') to convert type

X_train_tfidf = tfidf.fit_transform(train['sentence'].values.astype('U'))
X_test_tfidf = tfidf.transform(test['sentence'].values.astype('U'))

# v = TfidfVectorizer(decode_error='replace', encoding='utf-8')
# x = v.fit_transform(df['Review'])
# x = v.fit_transform(df['Review'].values.astype('U'))

In [0]:
X_train_tfidf.shape

(22022, 16901)

In [0]:
# EXCLUED_COLS = ['sentence', 'type']
# static_cols = [c for c in train.columns if not c in EXCLUED_COLS]
# X_train_static = train[static_cols].values
# X_test_static = test[static_cols].values

In [0]:
# train[static_cols].head()

In [0]:
from scipy.sparse import hstack, csr_matrix, vstack

In [0]:
X_train = hstack([X_train_tfidf, csr_matrix(X_train_static)]).tocsr()
X_test = hstack([X_test_tfidf, csr_matrix(X_test_static)]).tocsr()

In [0]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [0]:
clfs = {
    "RF": RandomForestClassifier(n_estimators=200, min_samples_leaf=2, max_depth=7, max_features=0.7, random_state=42, n_jobs=-1),
    "GB": GradientBoostingClassifier(n_estimators=200, min_samples_split=2, max_depth=5, max_features=0.7, random_state=111),
    "LR": LogisticRegression(penalty='l2', C=1.0),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(C=5, gamma=0.5)
}

In [0]:
for name, clf in clfs.items():
    print(name)
    y_true = train['type'].astype('int')
    clf.fit(X_train, y_true)
    y_pred = clf.predict(X_train)
    
    print("F1", f1_score(y_true, y_pred, average="micro"))
    print("CM", confusion_matrix(y_true, y_pred))
    print("ACC", accuracy_score(y_true, y_pred))
    
    y_true = test['type'].astype('int')
    y_pred = clf.predict(X_test)
    
    print("F1", f1_score(y_true, y_pred, average="micro"))
    print("CM", confusion_matrix(y_true, y_pred))
    print("ACC", accuracy_score(y_true, y_pred))

RF
F1 0.766188357097448
CM [[ 3167    21  1621]
 [  878   163  1983]
 [  617    29 13543]]
ACC 0.766188357097448
F1 0.7309811400720493
CM [[1236   15  609]
 [ 516   49 1109]
 [ 242   48 5614]]
ACC 0.7309811400720492
GB
F1 0.8474253019707565
CM [[ 3966   145   698]
 [  700   965  1359]
 [  348   110 13731]]
ACC 0.8474253019707565
F1 0.765310447128629
CM [[1415  118  327]
 [ 598  258  818]
 [ 187  167 5550]]
ACC 0.765310447128629
LR


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


F1 0.8501498501498501
CM [[ 4195    97   517]
 [  793   811  1420]
 [  345   128 13716]]
ACC 0.8501498501498501
F1 0.776541640178004
CM [[1515   87  258]
 [ 630  230  814]
 [ 183  137 5584]]
ACC 0.7765416401780039
KNN
F1 0.7740895468168195
CM [[ 2645   823  1341]
 [  347  1363  1314]
 [  306   844 13039]]
ACC 0.7740895468168195
F1 0.655541428268701
CM [[ 674  378  808]
 [ 258  396 1020]
 [ 184  603 5117]]
ACC 0.655541428268701
SVM
F1 0.9725728816637907
CM [[ 4709    28    72]
 [   64  2608   352]
 [   36    52 14101]]
ACC 0.9725728816637907
F1 0.7664759482941301
CM [[1444  202  214]
 [ 599  373  702]
 [ 209  278 5417]]
ACC 0.7664759482941301


In [0]:
# try with large C value in SVC

print("SVM-rbf")
y_true = train['type'].astype('int')
clf = SVC(C=1000, gamma=0.01, probability=True)
clf.fit(X_train, y_true)
y_pred = clf.predict(X_train)

print("F1", f1_score(y_true, y_pred, average="micro"))
print("CM", confusion_matrix(y_true, y_pred))
print("ACC", accuracy_score(y_true, y_pred))

y_true = test['type'].astype('int')
y_pred = clf.predict(X_test)

print("F1", f1_score(y_true, y_pred, average="micro"))
print("CM", confusion_matrix(y_true, y_pred))
print("ACC", accuracy_score(y_true, y_pred))

SVM-rbf
F1 0.9721641994369267
CM [[ 4718    29    62]
 [   75  2591   358]
 [   34    55 14100]]
ACC 0.9721641994369267
F1 0.7412587412587412
CM [[1361  274  225]
 [ 604  424  646]
 [ 300  393 5211]]
ACC 0.7412587412587412


In [0]:
# Try with fold
# Not yet install

# from pystacknet.pystacknet import StackNetClassifier

# model = StackNetClassifier(
#     models, metric="accuracy", 
#     folds=5,
#     restacking=False, 
#     use_retraining=True, 
#     use_proba=True, 
#     random_state=12345, n_jobs=1, verbose=1
# )

# y_train = train['label'].astype('int')
# model.fit(X_train, y_train)
# preds=model.predict_proba(X_test)

ModuleNotFoundError: ignored

In [0]:
import pickle

In [0]:
with open('NLP1920_random_forest.pkl', 'wb') as f:
  pickle.dump(clfs["RF"], f)

with open('NLP1920_gradient_boosting.pkl', 'wb') as f:
  pickle.dump(clfs["GB"], f)

with open('NLP1920_linear_regression.pkl', 'wb') as f:
  pickle.dump(clfs["LR"], f)

with open('NLP1920_svm.pkl', 'wb') as f:
  pickle.dump(clfs["SVM"], f)

In [0]:
!cp NLP1920_random_forest.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'
!cp NLP1920_gradient_boosting.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'
!cp NLP1920_linear_regression.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'
!cp NLP1920_svm.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'

In [0]:
with open('NLP1920_tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [0]:
!cp NLP1920_tfidf.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'

In [0]:
# Test model
# Try with Gradient Boosting first

test_sentence = 'Giao hàng nhanh, đóng gói cẩn thận. Son đánh lên rất mượt thích hơn các ver trước mình sử sụng. Rất thích nhé shop'
sen_tfidf = tfidf.transform([test_sentence])

In [0]:
feat = hstack([sen_tfidf]).tocsr()

In [0]:
label = clfs["GB"].predict(feat)

In [0]:
convertToTypeDict = {1: 'NEG', 2: 'NEU', 3: 'POS'}
print(convertToTypeDict[int(label[0])])

POS


In [0]:
# !cp test.csv drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'
# !cp train.csv drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'